In [1]:
import sys
sys.path.append("../")
import pandas as pd
import datetime as dt
from api.oanda_api import OandaApi
from dateutil import parser
import timeit
import plotly.graph_objects as go
import numpy as np
from technicals import trend
from technicals import zone
from technicals import pattern
from technicals import bottom
from technicals import candle
from charting import draw
from scipy.signal import argrelextrema
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [ ]:
def detect_setup(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips
):
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Bottom detection
    df['is_bottom'] = (
        (df['mid_l'] == df['mid_l'].rolling(window=rolling_window).min()) &
        (df['in_downtrend'] == True)
    )
    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'

    # Step 2: Track active zone
    df['active_zone_low'] = None
    df['active_zone_high'] = None

    current_low = None
    current_high = None
    last_bottom_idx = None
    breakout_found = False
    reentry_found = False
    confirmation_found = False

    for i in range(len(df)):
        if df.at[i, 'is_bottom']:
            # New setup: reset everything
            current_low = df.at[i, 'mid_l']
            current_high = df.at[i, 'mid_h']
            last_bottom_idx = i
            breakout_found = False
            reentry_found = False
            confirmation_found = False

        df.at[i, 'active_zone_low'] = current_low
        df.at[i, 'active_zone_high'] = current_high

        # Skip if setup was invalidated
        if current_high is None:
            continue

        max_allowed_low = current_high + breakout_threshold / 10000.0

        # Invalidate if price goes too far from zone
        if breakout_found and df.at[i, 'mid_l'] > max_allowed_low:
            current_low = None
            current_high = None
            last_bottom_idx = None
            breakout_found = False
            reentry_found = False
            confirmation_found = False
            df.at[i, 'active_zone_low'] = None
            df.at[i, 'active_zone_high'] = None
            continue

        # Step 2: Breakout
        if not breakout_found and df.at[i, 'mid_l'] > current_high:
            df.at[i, 'setup_stage'] = 'breakout'
            breakout_found = True
            continue  # Reentry can't be on breakout candle

        # Step 3: Reentry
        if breakout_found and not reentry_found:
            if current_low <= df.at[i, 'mid_l'] <= current_high:
                df.at[i, 'setup_stage'] = 'reentry'
                reentry_found = True
                continue  # Confirmation comes after reentry

        # Step 4: Confirmation (with spacing from bottom)
        if breakout_found and reentry_found and not confirmation_found:
            if df.at[i, strength_col] > strength_threshold:
                if i - last_bottom_idx >= 6:  # ✅ Enforce minimum spacing from bottom
                    df.at[i, 'setup_stage'] = 'confirmation'
                    confirmation_found = True

    return df

def detect_bottom_reversal_setups(
    df,
    strength_col='bullish_strength_score',
    strength_threshold=0.7,
    lookahead=25,
    proximity_pips=0.0030,
    rolling_window=40,
    breakout_threshold=20  # in pips (e.g. 30 = 0.0030)
):
    """
    Detects bottom reversal setups using new lows that occur only during downtrends.

    A setup is:
    1. A bottom candle (lowest in a window, while in_downtrend is True)
    2. A breakout candle (low + high > zone high)
    3. A reentry candle (low reenters the zone but never breaks below zone low)
    4. A strong bullish candle (close near zone high and strength score high)

    The setup is invalidated if any candle after reentry has a low greater than
    breakout_threshold above the zone high.

    Returns:
        DataFrame with setup_stage column added.
    """
    df = df.copy().reset_index(drop=True)
    df['setup_stage'] = None

    # Step 1: Tag bottom candles
    
    df.loc[df['is_bottom'], 'setup_stage'] = 'bottom'
    bottom_indexes = df.index[df['is_bottom']].tolist()

    for i in bottom_indexes:
        # 🔥 Always reset the active zone on every new bottom
        active_zone_low = df.at[i, 'mid_l']
        active_zone_high = df.at[i, 'mid_h']
        active_bottom_idx = i

        # Step 2: Look for breakout
        breakout_idx = None
        for j in range(i + 1, len(df)):
            row_j = df.iloc[j]
            if row_j['mid_l'] < active_zone_low:
                break  # Setup invalidated by lower low
            if row_j['mid_l'] > active_zone_high and row_j['mid_h'] > active_zone_high:
                breakout_idx = j
                break

        if breakout_idx is None or breakout_idx - i > lookahead:
            continue

        # Step 3: Reentry
        reentry_idx = None
        for k in range(breakout_idx + 1, breakout_idx + lookahead):
            if k >= len(df):
                break
            row_k = df.iloc[k]
            if active_zone_low <= row_k['mid_l'] <= active_zone_high:
                reentry_idx = k
                break
            if row_k['mid_l'] < active_zone_low:
                break  # Invalidated by new low

        if reentry_idx is None:
            continue

        # Step 4: Confirmation + breakout distance check
        confirmation_found = False
        breakout_distance_limit = active_zone_high + breakout_threshold / 10000.0
        df['breakout_distance_limit'] = breakout_distance_limit
        df['active_zone_high'] = active_zone_high
        df['breakout_threshold'] = breakout_threshold / 10000.0
        for m in range(reentry_idx + 1, reentry_idx + lookahead):
            if m >= len(df):
                break
            row_m = df.iloc[m]

            if row_m['mid_l'] < active_zone_low:
                break  # Invalidated by new low
            if row_m['mid_l'] > breakout_distance_limit:
                break  # Price went too far — invalidated

            if row_m[strength_col] >= strength_threshold:
                if abs(row_m['mid_l'] - active_zone_high) <= proximity_pips:
                    df.at[breakout_idx, 'setup_stage'] = 'breakout'
                    df.at[reentry_idx, 'setup_stage'] = 'reentry'
                    df.at[m, 'setup_stage'] = 'confirmation'
                    confirmation_found = True
                    break

        # No need to manually reset active_* vars — loop resets on next bottom

    return df

def find_support_resistance(df, price_col='mid_c', high_col='mid_h', low_col='mid_l', window=3, clustering_threshold=0.0050):
    """
    Identifies support and resistance levels in candlestick data.
    
    Args:
        df (pd.DataFrame): Your OHLC dataframe.
        price_col (str): Column name for close/mid price.
        high_col (str): Column name for highs.
        low_col (str): Column name for lows.
        window (int): Lookback window to detect local highs/lows.
        clustering_threshold (float): Maximum distance between levels to consider them the same zone.

    Returns:
        Tuple[List[float], List[float]]: (support_levels, resistance_levels)
    """
    
    local_min_idx = argrelextrema(df[low_col].values, np.less_equal, order=window)[0]
    local_max_idx = argrelextrema(df[high_col].values, np.greater_equal, order=window)[0]

    raw_supports = df.iloc[local_min_idx][low_col].values
    raw_resistances = df.iloc[local_max_idx][high_col].values

    def cluster_levels(levels):
        clustered = []
        levels = sorted(levels)
        for level in levels:
            if not clustered:
                clustered.append([level])
            elif abs(level - np.mean(clustered[-1])) <= clustering_threshold:
                clustered[-1].append(level)
            else:
                clustered.append([level])
        return [round(np.mean(group), 5) for group in clustered if len(group) >= 2]  # Only return stronger levels

    support_levels = cluster_levels(raw_supports)
    resistance_levels = cluster_levels(raw_resistances)

    return support_levels, resistance_levels

def get_zones_for_price(price, support_levels, resistance_levels, num_of_zones=3, min_gap=0.0, min_width=0.0015):
    """
    Returns non-overlapping (support, resistance) zones where the support is above the given price,
    and there's at least `min_gap` space and `min_width` size.

    Args:
        price (float): Current price.
        support_levels (list of float): Detected support levels.
        resistance_levels (list of float): Detected resistance levels.
        num_of_zones (int): Number of zones to return.
        min_gap (float): Minimum gap between zones.
        min_width (float): Minimum acceptable width of a zone.

    Returns:
        List of tuples: [(support1, resistance1), (support2, resistance2), ...]
    """

    support_levels = sorted(support_levels)
    resistance_levels = sorted(resistance_levels)

    zones = []
    last_resistance = price

    sup_above = [s for s in support_levels if s > price]

    for support in sup_above:
        if support <= last_resistance + min_gap:
            continue

        possible_resistances = [r for r in resistance_levels if r > support]
        for resistance in possible_resistances:
            width = resistance - support
            if width >= min_width:
                zones.append((support, resistance))
                last_resistance = resistance
                break  # Move on to the next zone

        if len(zones) == num_of_zones:
            break

    return zones

def attach_zones_to_confirmations(
    df,
    window=3,
    clustering_threshold=0.0050,
    num_of_zones=3
):
    """
    For each confirmation candle:
        - Attach relevant support/resistance zones based on past data only
        - Compute zone-to-stop-loss ratio using second zone
        - Add 'confirmation_zones', 'zone_sl_ratio', and 'meets_ratio' columns
    """
    from copy import deepcopy
    df['confirmation_zones'] = None
    df['zone_sl_ratio'] = None
    df['meets_ratio'] = False

    for i in range(len(df)):
        if df.at[i, 'setup_stage'] == 'confirmation':
            past_df = df.iloc[:i]
            if len(past_df) < window * 2:
                continue

            support_levels, resistance_levels = find_support_resistance(
                past_df,
                price_col='mid_c',
                high_col='mid_h',
                low_col='mid_l',
                window=window,
                clustering_threshold=clustering_threshold
            )

            current_price = df.at[i, 'mid_c']
            current_low = df.at[i, 'mid_l']

            zones = get_zones_for_price(
                price=current_price,
                support_levels=support_levels,
                resistance_levels=resistance_levels,
                num_of_zones=num_of_zones
            )

            df.at[i, 'confirmation_zones'] = deepcopy(zones)

            if len(zones) >= 2:
                zone_top = zones[1][1]  # Top of second zone (resistance)
                reward = zone_top - current_price
                risk = current_price - current_low

                if risk > 0:
                    ratio = reward / risk
                    df.at[i, 'zone_sl_ratio'] = round(ratio, 3)
                    df.at[i, 'meets_ratio'] = ratio >= 1.0

def simulate_trades(df, sl_pips=15, pair='EUR_USD'):
    pip_divisor = 100 if 'JPY' in pair else 10000
    pip_size = sl_pips / pip_divisor

    df['trade'] = None
    df['entry_price'] = None
    df['stop_loss'] = None
    df['take_profit'] = None
    df['pips'] = None

    active_trade = None

    for i in range(len(df)):
        row = df.iloc[i]

        # Check for new confirmation candle to enter trade
        if row['setup_stage'] == 'confirmation':
            zone = row.get('zone')
            if not isinstance(zone, tuple) or len(zone) != 2:
                continue

            zone_low = zone[0]
            entry_price = row['mid_c']
            stop_loss = zone_low - pip_size
            take_profit = entry_price + ((entry_price - stop_loss) * 2)

            active_trade = {
                'entry_idx': i,
                'entry_price': entry_price,
                'stop_loss': stop_loss,
                'take_profit': take_profit
            }

            df.at[df.index[i], 'trade'] = 'opened'
            df.at[df.index[i], 'entry_price'] = entry_price
            df.at[df.index[i], 'stop_loss'] = stop_loss
            df.at[df.index[i], 'take_profit'] = take_profit
            continue

        # If trade is active, check for SL/TP hit
        if active_trade:
            high = row['mid_h']
            low = row['mid_l']
            idx = df.index[i]

            if low <= active_trade['stop_loss']:
                df.at[idx, 'trade'] = 'closed - sl'
                df.at[idx, 'pips'] = round((active_trade['stop_loss'] - active_trade['entry_price']) / (1 / pip_divisor), 1)
                active_trade = None
            elif high >= active_trade['take_profit']:
                df.at[idx, 'trade'] = 'closed - tp'
                df.at[idx, 'pips'] = round((active_trade['take_profit'] - active_trade['entry_price']) / (1 / pip_divisor), 1)
                active_trade = None

def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend.apply_downtrend(df)
    bottom.apply_bottom_zones(df)
    zone.apply_zone_exits_and_reentries(df, 50, 'EUR_USD')
    candle.detect_strong_bullish(df)
    #need to make this not return a df but add in place
    df = detect_setup(df)
    attach_zones_to_confirmations(df)
    return df

def plot_trades(df, title="Trade Zones & Entries"):
    fig = go.Figure()

    y_min = df['mid_l'].min()
    y_max = df['mid_h'].max()
    padding = (y_max - y_min) * 0.1

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=df['sTime'],
        open=df['mid_o'],
        high=df['mid_h'],
        low=df['mid_l'],
        close=df['mid_c'],
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B', 
        decreasing_fillcolor='#CC2E3C',
        increasing_line_color='#24A06B',
        decreasing_line_color='#FF3A4C',
        name="Candles"
    ))

    # Add MAs
    # fig.add_trace(go.Scatter(
    #     x=df['sTime'],
    #     y=df['ma_10'],
    #     line=dict(width=2),
    #     line_shape='spline',
    #     name='MA_10'
    # ))
    # fig.add_trace(go.Scatter(
    #     x=df['sTime'],
    #     y=df['ma_150'],
    #     line=dict(width=2),
    #     line_shape='spline',
    #     name='MA_150'
    # ))

    # Mark confirmation (entry) candles
    entry_df = df[df['setup_stage'] == 'confirmation']
    fig.add_trace(go.Candlestick(
        x=entry_df['sTime'],
        open=entry_df['mid_o'],
        high=entry_df['mid_h'],
        low=entry_df['mid_l'],
        close=entry_df['mid_c'],
        line=dict(width=2), opacity=1,
        increasing_fillcolor='yellow',
        increasing_line_color='yellow'
        
    ))

    # Mark bottom candles
    bottom_df = df[df['setup_stage'] == 'bottom']
    fig.add_trace(go.Scatter(
        x=bottom_df['sTime'],
        y=bottom_df['mid_l'],
        mode='markers',
        marker=dict(color='green', size=15, symbol='triangle-up'),
        name='Bottm',
        showlegend=True
    ))

    # Mark reentry candles
    # reentry_df = df[df['setup_stage'] == 'reentry']
    # fig.add_trace(go.Scatter(
    #     x=reentry_df['sTime'],
    #     y=reentry_df['mid_h'],
    #     mode='markers',
    #     marker=dict(color='blue', size=15, symbol='triangle-down'),
    #     name='Reentry',
    #     showlegend=True
    # ))

    # for i, row in entry_df.iterrows():
    #     zones = row['confirmation_zones']  # This should be a list of tuples
    #     time = row['sTime']

    #     for zone in zones:
    #         low, high = zone
    #         fig.add_trace(go.Scatter(
    #             x=[time, time],
    #             y=[low, high],
    #             mode='lines',
    #             line=dict(color='yellow', width=2, dash='dot'),
    #             name='Zone'  # Show legend only once
    #         ))

    # Mark downtrend candles
    # downtrend_df = df[df['in_downtrend'] == True]
    # fig.add_trace(go.Candlestick(
    #     x=downtrend_df['sTime'],
    #     open=downtrend_df['mid_o'],
    #     high=downtrend_df['mid_h'],
    #     low=downtrend_df['mid_l'],
    #     close=downtrend_df['mid_c'],
    #     line=dict(width=1), opacity=1,
    #     increasing_fillcolor='yellow',
    #     increasing_line_color='yellow',
    #     decreasing_line_color='yellow',
    #     name="Candles"
    # ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Price',
        width=2000,
        height=1000,
        hovermode='x unified',
        margin=dict(l=10, r=10, b=10, t=30),
        paper_bgcolor="#2c303c",
        plot_bgcolor="#2c303c",
        font=dict(size=10, color="#e1e1e1"),
        yaxis=dict(
            range=[y_min - padding, y_max + padding],
            fixedrange=False  # Allow zooming
        )
    )

    fig.update_xaxes(
        gridcolor="#1f292f",
        rangeslider=dict(visible=True),
        nticks=5
    )

    fig.update_yaxes(
        gridcolor="#1f292f"
    )

    fig.show()

def filter_df_by_date(df, start, end):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    return df[(df['time'] >= start) & (df['time'] <= end)]

def summarize_trades(df, starting_balance=10000, risk_per_trade=0.02):

    # Filter only closed trades
    closed_trades = df[df['trade'].isin(['closed - sl', 'closed - tp'])].copy()

    if closed_trades.empty:
        print("No closed trades found.")
        return

    # Calculate metrics
    total_trades = len(closed_trades)
    total_pips = closed_trades['pips'].sum()
    win_rate = (closed_trades['trade'] == 'closed - tp').mean() * 100

    # Cumulative profit over time (simulate compounding returns with 2% risk)
    balance = starting_balance
    balances = [balance]

    for i, row in closed_trades.iterrows():
        risk_amount = balance * risk_per_trade
        if row['trade'] == 'closed - sl':
            balance -= risk_amount
        elif row['trade'] == 'closed - tp':
            reward = risk_amount * (row['pips'] / abs(row['pips']) if row['pips'] != 0 else 1)
            balance += reward
        balances.append(balance)

    closed_trades['cumulative_pips'] = closed_trades['pips'].cumsum()
    closed_trades['cumulative_balance'] = balances[1:]
    closed_trades['time'] = pd.to_datetime(closed_trades['time'])

    total_dollar_gain = balances[-1] - starting_balance
    percent_gain = (balances[-1] / starting_balance - 1) * 100

    # Print summary
    print(f"📈 Total Trades: {total_trades}")
    print(f"💰 Total Pips: {round(total_pips, 1)}")
    print(f"✅ Win Rate: {round(win_rate, 2)}%")
    print(f"💵 Total $ Gain: ${round(total_dollar_gain, 2)}")
    print(f"📊 Percent Gain: {round(percent_gain, 2)}%")

    # Plot Pips and Balance
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=closed_trades['time'],
        y=closed_trades['cumulative_pips'],
        mode='lines+markers',
        name='Cumulative Pips',
        line=dict(width=2),
        marker=dict(size=4)
    ))

    fig.add_trace(go.Scatter(
        x=closed_trades['time'],
        y=closed_trades['cumulative_balance'],
        mode='lines+markers',
        name='Account Balance ($)',
        line=dict(width=2),
        marker=dict(size=4),
        yaxis="y2"
    ))

    fig.update_layout(
        title="📈 Strategy Performance",
        xaxis_title="Time",
        yaxis=dict(title="Cumulative Pips"),
        yaxis2=dict(title="Account Balance ($)", overlaying='y', side='right'),
        template="plotly_white",
        hovermode="x unified",
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

In [ ]:
df = pd.read_pickle('../data/EUR_USD_H1.pkl')
# df = filter_df_by_date(df, "2024-01-01T00:00:00Z", "2025-01-01T00:00:00Z")

df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
trend.apply_downtrend(df)
bottom.apply_bottom_zones(df)
zone.apply_zone_exits_and_reentries(df, 50, 'EUR_USD')
candle.detect_strong_bullish(df)
simulate_trades(df)
# df = apply_technicals(df)
# df = simulate_trades(df)
# df_between.head(45)

In [ ]:
fig = draw.draw_candlestick_chart(df)


# draw.highlight_downtrend_candles(fig, df)
# draw.highlight_bottom_zones(fig, df)
# draw.highlight_exits_and_reentries(fig, df)
draw.highlight_strong_bullish_candles(fig, df)

fig.show()

In [47]:
df = pd.read_pickle('../backtesting/results/EUR_USD_H1_analyzed.pkl')
print("Opened trades:", (df['trade'] == 'opened').sum())

def extract_trades(df, context_candles=30):
    trades = []

    # Get indices where a trade was opened
    open_indices = df.index[df['trade'] == 'opened'].tolist()

    for entry_idx in open_indices:
        # Find the close of this trade
        subsequent_df = df.loc[entry_idx:]
        close_idx = subsequent_df[subsequent_df['trade'].isin(['closed - sl', 'closed - tp'])].index

        if close_idx.empty:
            continue  # Skip if no close found
        close_idx = close_idx[0]

        # Walk back to find the most recent bottom zone before the entry
        entry_pos = df.index.get_loc(entry_idx)
        bottom_pos = None
        for i in range(entry_pos, -1, -1):
            if df.iloc[i]['setup_stage'] == 'bottom':
                bottom_pos = i
                break

        if bottom_pos is None:
            continue  # No bottom zone found

        start_pos = max(0, bottom_pos - context_candles)
        start_idx = df.index[start_pos]

        trade_df = df.loc[start_idx:close_idx].copy()
        trades.append(trade_df)

    return trades

# Confirm updated version
"Updated trade extraction function with bottom zone context is ready."



Opened trades: 50


'Updated trade extraction function with bottom zone context is ready.'

In [ ]:
df_trades = extract_trades(df)
# plot_trades(df_trades[0])
# df_trades[0].head(50)


IndexError: list index out of range

0


In [28]:
df = pd.read_pickle('../backtesting/results/EUR_USD_H1_analyzed.pkl')
# df = df[df['trade'] == 'opened']
df.tail(80)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,sTime,ma_10,ma_150,in_downtrend,is_bottom,zone,setup_stage,body,total_range,upper_wick,lower_wick,wick_ratio,avg_range,range_ok,wick_ok,close_near_high,strong_bullish,trade,entry_price,stop_loss,take_profit,pips
62110,2024-12-26 14:00:00+00:00,6273,1.04028,1.04046,1.03951,1.04024,1.04020,1.04037,1.03943,1.04016,1.04035,1.04054,1.03959,1.04032,s24-12-26 14:00,1.039961,1.041835,True,False,"(1.0343, 1.03775)",None,-0.00004,0.00095,0.00018,0.00073,-0.042105,0.000705,False,False,False,False,None,None,None,None,None
62111,2024-12-26 15:00:00+00:00,7060,1.04024,1.04152,1.03982,1.04088,1.04015,1.04144,1.03973,1.04081,1.04032,1.04159,1.03989,1.04096,s24-12-26 15:00,1.040071,1.041784,True,False,"(1.0343, 1.03775)",None,0.00064,0.00170,0.00064,0.00042,0.376471,0.000753,True,False,False,False,None,None,None,None,None
62112,2024-12-26 16:00:00+00:00,4989,1.04089,1.04228,1.04083,1.04220,1.04082,1.04221,1.04075,1.04213,1.04096,1.04236,1.04090,1.04228,s24-12-26 16:00,1.040308,1.041740,True,False,"(1.0343, 1.03775)",None,0.00131,0.00145,0.00008,0.00006,0.903448,0.000767,True,True,True,True,None,None,None,None,None
62113,2024-12-26 17:00:00+00:00,4183,1.04222,1.04227,1.04065,1.04118,1.04215,1.04220,1.04057,1.04110,1.04229,1.04235,1.04073,1.04126,s24-12-26 17:00,1.040448,1.041687,True,False,"(1.0343, 1.03775)",None,-0.00104,0.00162,0.00005,0.00053,-0.641975,0.000788,True,False,False,False,None,None,None,None,None
62114,2024-12-26 18:00:00+00:00,4836,1.04118,1.04300,1.04118,1.04210,1.04110,1.04292,1.04110,1.04201,1.04125,1.04309,1.04125,1.04218,s24-12-26 18:00,1.040708,1.041633,True,False,"(1.0343, 1.03775)",None,0.00092,0.00182,0.00090,0.00000,0.505495,0.000848,True,False,False,False,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62185,2024-12-31 17:00:00+00:00,8546,1.03505,1.03550,1.03439,1.03486,1.03498,1.03543,1.03432,1.03478,1.03512,1.03558,1.03446,1.03493,s24-12-31 17:00,1.038762,1.040979,True,True,"(1.03439, 1.0355)",None,-0.00019,0.00111,0.00045,0.00047,-0.171171,0.001236,False,False,False,False,None,None,None,None,None
62186,2024-12-31 18:00:00+00:00,6355,1.03484,1.03568,1.03455,1.03531,1.03476,1.03560,1.03447,1.03524,1.03491,1.03576,1.03463,1.03538,s24-12-31 18:00,1.038109,1.040955,True,False,"(1.03439, 1.0355)",None,0.00047,0.00113,0.00037,0.00029,0.415929,0.001271,False,False,False,False,None,None,None,None,None
62187,2024-12-31 19:00:00+00:00,4853,1.03530,1.03649,1.03518,1.03548,1.03522,1.03641,1.03510,1.03540,1.03538,1.03657,1.03526,1.03556,s24-12-31 19:00,1.037470,1.040920,True,False,"(1.03439, 1.0355)",None,0.00018,0.00131,0.00101,0.00012,0.137405,0.001302,False,False,False,False,None,None,None,None,None
62188,2024-12-31 20:00:00+00:00,2870,1.03550,1.03632,1.03546,1.03585,1.03542,1.03624,1.03537,1.03578,1.03557,1.03640,1.03553,1.03592,s24-12-31 20:00,1.036926,1.040896,True,False,"(1.03439, 1.0355)",None,0.00035,0.00086,0.00047,0.00004,0.406977,0.001300,False,False,False,False,None,None,None,None,None


In [25]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY"]
granularities = ['H1']

for pair in pairs:
    for g in granularities:
        print(f"{pair} {g}")
        df = pd.read_pickle(f"../backtesting/results/{pair}_{g}_analyzed.pkl")
        summarize_trades(df)

AUD_USD H1
📈 Total Trades: 63
💰 Total Pips: -687.2
✅ Win Rate: 44.44%
💵 Total $ Gain: $-1415.45
📊 Percent Gain: -14.15%


EUR_USD H1
📈 Total Trades: 44
💰 Total Pips: 1258.6
✅ Win Rate: 63.64%
💵 Total $ Gain: $2601.49
📊 Percent Gain: 26.01%


GBP_USD H1
📈 Total Trades: 65
💰 Total Pips: -548.5
✅ Win Rate: 47.69%
💵 Total $ Gain: $-704.09
📊 Percent Gain: -7.04%


USD_CHF H1
📈 Total Trades: 66
💰 Total Pips: -417.7
✅ Win Rate: 45.45%
💵 Total $ Gain: $-1247.26
📊 Percent Gain: -12.47%


USD_JPY H1
📈 Total Trades: 50
💰 Total Pips: -67.0
✅ Win Rate: 54.0%
💵 Total $ Gain: $725.17
📊 Percent Gain: 7.25%


In [3]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY"]
granularities = ['M5', 'M30', 'H1', 'H4']

for pair in pairs:
    for g in granularities:
        print(f"{pair} {g}")
        df = pd.read_pickle(f"../backtesting/results/{pair}_{g}_analyzed.pkl")
        summarize_trades(df)

AUD_USD M5
📈 Total Trades: 533
💰 Total Pips: 3104.3
✅ Win Rate: 54.78%
💵 Total $ Gain: $14930.7
📊 Percent Gain: 149.31%


AUD_USD M30
📈 Total Trades: 148
💰 Total Pips: 1771.5
✅ Win Rate: 59.46%
💵 Total $ Gain: $6997.29
📊 Percent Gain: 69.97%


AUD_USD H1
📈 Total Trades: 63
💰 Total Pips: -687.2
✅ Win Rate: 44.44%
💵 Total $ Gain: $-1415.45
📊 Percent Gain: -14.15%


AUD_USD H4
📈 Total Trades: 8
💰 Total Pips: 85.5
✅ Win Rate: 50.0%
💵 Total $ Gain: $-15.99
📊 Percent Gain: -0.16%


EUR_USD M5
📈 Total Trades: 590
💰 Total Pips: 3275.4
✅ Win Rate: 54.24%
💵 Total $ Gain: $14159.91
📊 Percent Gain: 141.6%


...

USD_CHF H4
📈 Total Trades: 9
💰 Total Pips: 109.0
✅ Win Rate: 55.56%
💵 Total $ Gain: $183.69
📊 Percent Gain: 1.84%


USD_JPY M5
📈 Total Trades: 533
💰 Total Pips: 1626.2
✅ Win Rate: 52.91%
💵 Total $ Gain: $6710.65
📊 Percent Gain: 67.11%


USD_JPY M30
📈 Total Trades: 106
💰 Total Pips: 2664.2
✅ Win Rate: 61.32%
💵 Total $ Gain: $5822.69
📊 Percent Gain: 58.23%


USD_JPY H1
📈 Total Trades: 50
💰 Total Pips: -67.0
✅ Win Rate: 54.0%
💵 Total $ Gain: $725.17
📊 Percent Gain: 7.25%


USD_JPY H4
📈 Total Trades: 3
💰 Total Pips: -401.6
✅ Win Rate: 0.0%
💵 Total $ Gain: $-588.08
📊 Percent Gain: -5.88%


In [ ]:
pairs = ["AUD_USD", "EUR_USD", "GBP_USD", "USD_CHF", "USD_JPY", "NZD_USD", "USD_CAD"]
for pair in pairs:
    print(pair)
    summarize_trades(f"../backtesting/results/{pair}_H1_analyzed.pkl")

In [ ]:
df_an = pd.read_pickle('../data/GBP_USD_H4.pkl').copy()
def apply_technicals(df):
    df['sTime'] = [dt.datetime.strftime(x, "s%y-%m-%d %H:%M") for x in df.time]
    trend.apply_downtrend(df)
    df['bullish_strength_score'] = df.apply(pattern.bullish_strength, axis=1)
    #need to make this not return a df but add in place
    df = pattern.detect_bottom_reversal_setups(df)
    zone.attach_zones_to_confirmations(df)
    return df

df_an = apply_technicals(df_an)
df_an.head(40)